<a href="https://colab.research.google.com/github/enitin-agarwal/azureforaws/blob/master/MimicML_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, Embedding, LSTM, GlobalAveragePooling1D
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans
import networkx as nx
!pip install torch_geometric
!pip install torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data








In [ ]:
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')
# Initialize BigQuery Client
client = bigquery.Client(project='physionet-data')
# load the dataset list
datasets = client.list_datasets()

Authenticated


In [ ]:
 %%bigquery --project ultimate-balm-449016-q6

# Load new datasets (MIMIC IV, eICU)
SELECT * from `physionet-data.mimiciii_demo.admissions`

#admissions = pd.read_csv('ADMISSIONS.csv')
#patients = pd.read_csv('PATIENTS.csv')
#events = pd.read_csv('EVENTS.csv')


Query is running:   0%|          |

Downloading:   0%|          |

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
1,12280,10029,132349,2139-09-22 10:58:00,2139-10-02 14:29:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,PROTESTANT QUAKER,DIVORCED,WHITE,2139-09-22 06:03:00,2139-09-22 11:50:00,SYNCOPE;TELEMETRY,0,1
2,12283,10033,157235,2132-12-05 02:46:00,2132-12-08 15:15:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,POLI,CATHOLIC,MARRIED,WHITE,2132-12-04 20:11:00,2132-12-05 04:05:00,RENAL FAILIURE-SYNCOPE-HYPERKALEMIA,0,1
3,12288,10038,111115,2144-02-09 17:53:00,2144-02-21 13:30:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,ENGL,CATHOLIC,WIDOWED,WHITE,2144-02-09 13:35:00,2144-02-09 20:12:00,FAILURE TO THRIVE,0,1
4,12293,10043,168674,2185-04-14 00:23:00,2185-04-26 18:20:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,CATHOLIC,None,WHITE,2185-04-13 23:01:00,2185-04-14 02:56:00,RESPIRATORY DISTRESS,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,12311,10061,145203,2107-01-16 11:33:00,2107-02-10 11:30:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,None,CATHOLIC,WIDOWED,WHITE,NaT,NaT,CHEST PAIN/ CATH,0,1
125,41087,44212,163189,2123-11-24 14:14:00,2123-12-30 14:31:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,ENGL,UNOBTAINABLE,SINGLE,BLACK/AFRICAN AMERICAN,NaT,NaT,ACUTE RESPIRATORY DISTRESS SYNDROME;ACUTE RENA...,0,1
126,39869,40310,186361,2144-07-11 15:02:00,2144-11-12 14:40:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Private,ENGL,CATHOLIC,SINGLE,WHITE,NaT,NaT,FACIAL NUMBNESS,0,1
127,40379,41914,101361,2145-12-01 18:13:00,2145-12-18 17:45:00,NaT,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,REHAB/DISTINCT PART HOSP,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaT,NaT,ELEVATED LIVER FUNCTIONS;S/P LIVER TRANSPLANT,0,1


In [ ]:
admissions = _


In [ ]:
 %%bigquery --project ultimate-balm-449016-q6

SELECT * from `physionet-data.mimiciii_demo.patients`


Query is running:   0%|          |

Downloading:   0%|          |

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,9467,10006,F,2094-03-05,2165-08-12,2165-08-12,2165-08-12,1
1,9472,10011,F,2090-06-05,2126-08-28,2126-08-28,NaT,1
2,9474,10013,F,2038-09-03,2125-10-07,2125-10-07,2125-10-07,1
3,9478,10017,F,2075-09-21,2152-09-12,NaT,2152-09-12,1
4,9486,10026,F,1895-05-17,2195-11-24,NaT,2195-11-24,1
...,...,...,...,...,...,...,...,...
95,31771,43798,M,2136-07-29,2200-12-31,2200-12-31,2200-12-31,1
96,31790,43881,M,2051-03-24,2105-02-16,2105-02-16,2105-02-16,1
97,31838,44083,M,2057-11-15,2114-02-20,2114-02-20,2114-02-20,1
98,31853,44154,M,1878-05-14,2178-05-15,2178-05-15,2178-05-15,1


In [ ]:
patients = _


In [ ]:
 %%bigquery --project ultimate-balm-449016-q6

SELECT * from `physionet-data.mimiciii_demo.chartevents`


Query is running:   0%|          |

Downloading:   0%|          |

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,274486066,44212,163189,<NA>,224032,2124-11-30 20:00:00,2123-12-06 09:32:00,20030,"Gluteal, Left",NaN,None,0,1,None,None
1,274486067,44212,163189,<NA>,224032,2124-11-30 20:00:00,2123-12-06 09:32:00,20030,"Gluteal, Right",NaN,None,0,1,None,None
2,272826995,42199,178513,<NA>,220739,2117-03-23 00:00:00,2117-03-23 23:59:00,19056,None,1.0,None,0,1,None,None
3,272826996,42199,178513,<NA>,223753,2117-03-23 00:00:00,2117-03-23 23:59:00,19056,Calm/Cooperative,4.0,None,0,1,None,None
4,272826997,42199,178513,<NA>,223754,2117-03-23 00:00:00,2117-03-23 23:59:00,19056,Yes,NaN,None,0,1,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
758350,85815903,10013,165520,264446,742,2125-10-07 00:00:00,2125-10-07 04:29:00,14205,1,1.0,kg,<NA>,<NA>,None,NotStopd
758351,85815913,10013,165520,264446,742,2125-10-07 01:00:00,2125-10-07 04:29:00,14205,1,1.0,kg,<NA>,<NA>,None,NotStopd
758352,85815923,10013,165520,264446,742,2125-10-07 02:00:00,2125-10-07 04:29:00,14205,1,1.0,kg,<NA>,<NA>,None,NotStopd
758353,85815933,10013,165520,264446,742,2125-10-07 03:00:00,2125-10-07 04:29:00,14205,1,1.0,kg,<NA>,<NA>,None,NotStopd


In [ ]:
events = _

In [ ]:
%%bigquery --project ultimate-balm-449016-q6

SELECT * from `physionet-data.mimiciii_demo.d_items`

Query is running:   0%|          |

Downloading:   0%|          |

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],None,carevue,chartevents,None,None,None,None
1,458,498,PCA Lockout (Min),None,carevue,chartevents,None,None,None,None
2,459,499,PCA Medication,None,carevue,chartevents,None,None,None,None
3,460,500,PCA Total Dose,None,carevue,chartevents,None,None,None,None
4,461,501,PCV Exh Vt (Obser),None,carevue,chartevents,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
12482,12747,220283,Resistance,Resistance,metavision,chartevents,Respiratory,cmH2O/L/seconds,Numeric,None
12483,15511,228382,Total Peripheral Resistance Index (TPRI) (NICOM),TPRI (NICOM),metavision,chartevents,NICOM,dynes*sec/cm5/m2,Numeric,None
12484,15183,228183,SVRI (PiCCO)_OLD_1,SVRI (PiCCO)_OLD_1,metavision,chartevents,PiCCO,dynes.sec.cm-5/m2,Numeric,None
12485,15185,228185,SVRI (PiCCO),SVRI (PiCCO),metavision,chartevents,PiCCO,dynes.sec.cm-5/m2,Numeric,None


In [ ]:
items = _

In [ ]:
%%bigquery --project ultimate-balm-449016-q6

SELECT * from `physionet-data.mimiciii_notes.noteevents` limit 5000

Query is running:   0%|          |

Downloading:   0%|          |

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,894319,17868,124759,2132-11-29,2132-11-29 08:48:00,NaT,Radiology,CT HEAD W/O CONTRAST,<NA>,<NA>,[**2132-11-29**] 8:48 AM\n CT HEAD W/O CONTRAS...
1,562430,64334,191708,2201-03-05,2201-03-05 14:32:00,2201-03-05 14:32:26,Nursing,Nursing Progress Note,15654,<NA>,"Endocarditis, Bacterial\n Assessment:\n Ac..."
2,610295,53490,100499,2105-11-19,2105-11-19 01:07:00,2105-11-19 05:55:37,Nursing,Nursing Progress Note,21116,<NA>,This 25 yo woman who came in to ED with chest ...
3,1126365,74369,162363,2149-03-28,2149-03-28 04:56:00,NaT,Radiology,CT HEAD W/O CONTRAST,<NA>,<NA>,[**2149-3-28**] 4:56 AM\n CT HEAD W/O CONTRAST...
4,402770,16463,183203,2129-02-19,2129-02-18 23:43:00,2129-02-19 06:08:55,Nursing,Nursing Progress Note,14357,<NA>,Mr. [**Known lastname 11722**] is a 84-year-ol...
...,...,...,...,...,...,...,...,...,...,...,...
2083175,1738317,4432,140382,2121-08-28,2121-08-28 15:30:00,2121-08-28 15:35:00,Nursing/other,Report,16571,<NA>,NPN Days\n\n\n#3 FEN- TF=150cc/kg/d of PE or B...
2083176,1526363,20083,162595,2197-11-08,2197-11-08 06:38:00,2197-11-08 06:44:00,Nursing/other,Report,15805,<NA>,"NEURO; A&OX3, MAE, FOLLOWS COMMANDS\n\nRESPIR;..."
2083177,1314066,3965,134304,2121-10-18,2121-10-18 10:13:00,2121-10-18 10:15:00,Nursing/other,Report,19648,<NA>,Resp. Note\nPt was placed on SBt in early am w...
2083178,1948344,19505,159107,2125-11-28,2125-11-28 02:38:00,2125-11-28 02:49:00,Nursing/other,Report,17101,<NA>,NPN NIGHTS\n\n\nFEN:150cc/kg/d of EF and IVF. ...


In [ ]:
notes = _

In [ ]:
# Data preprocessing
# Merge datasets to create a unified patient record
data = pd.merge(admissions, patients, on='SUBJECT_ID', how='inner')
data = pd.merge(data, events, on='SUBJECT_ID', how='inner')

# Fill missing values
data.fillna(method='bfill', inplace=True)

<ipython-input-18-de5b28514555>:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)


In [ ]:
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import MultiLabelBinarizer
nltk.download('punkt_tab')

# Merge chartevents with D_ITEMS to extract heart rate and temperature
vitals_items = ['Heart Rate', 'Temperature']
item_ids = items[items['LABEL'].isin(vitals_items)]['ITEMID'].unique()

vitals = events[events['ITEMID'].isin(item_ids)]
vitals = vitals.merge(items[['ITEMID', 'LABEL']], on='ITEMID')
vitals_pivot = vitals.pivot_table(index=['SUBJECT_ID', 'CHARTTIME'], columns='LABEL', values='VALUENUM').reset_index()

# Extract discharge notes
discharge_notes = notes[notes['CATEGORY'] == 'Discharge summary'][['SUBJECT_ID', 'TEXT']]

# Basic NLP on notes: extract keywords and sentiment
nltk.download('punkt')
discharge_notes['keywords'] = discharge_notes['TEXT'].apply(lambda x: [word for word in word_tokenize(x.lower()) if word.isalnum()])

# Merge admissions and patients data
patient_info = admissions.merge(patients, on='SUBJECT_ID')

# Add diagnosis codes as one-hot features
diagnoses = pd.read_csv('DIAGNOSES_ICD.csv')
mlb = MultiLabelBinarizer()
patient_diagnoses = diagnoses.groupby('SUBJECT_ID')['ICD9_CODE'].apply(list).reset_index()
patient_diagnoses_encoded = pd.DataFrame(mlb.fit_transform(patient_diagnoses['ICD9_CODE']), columns=mlb.classes_)
patient_diagnoses = pd.concat([patient_diagnoses['SUBJECT_ID'], patient_diagnoses_encoded], axis=1)

# Merge everything into a single DataFrame
final_data = patient_info.merge(vitals_pivot, on='SUBJECT_ID', how='left')
final_data = final_data.merge(discharge_notes[['SUBJECT_ID', 'keywords']], on='SUBJECT_ID', how='left')
final_data = final_data.merge(patient_diagnoses, on='SUBJECT_ID', how='left')

# Fill missing values
final_data.fillna(0, inplace=True)

# Save the engineered dataset
final_data.to_csv('mimic_feature_engineered.csv', index=False)

print("Feature engineering completed! Data saved as mimic_feature_engineered.csv")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Feature Engineering Section
print("Performing Feature Engineering...")

# Extract static features
data['age'] = (pd.to_datetime(data['DISCHTIME']) - pd.to_datetime(data['DOB'])).dt.days // 365
static_features = ['age', 'GENDER', 'ADMISSION_TYPE', 'DIAGNOSIS']

# Time-series feature aggregation
#data['avg_heart_rate'] = data['HEART_RATE'].groupby(data['SUBJECT_ID']).transform('mean')
#data['max_temperature'] = data['TEMPERATURE'].groupby(data['SUBJECT_ID']).transform('max')

# Sentiment analysis on notes
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
data['sentiment'] = data['NOTES'].apply(lambda x: sia.polarity_scores(str(x))['compound'])

#print("Features engineered: age, avg_heart_rate, max_temperature, sentiment")
print("Features engineered: age, max_temperature, sentiment")

Performing Feature Engineering...


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


KeyError: 'NOTES'

In [ ]:
# New Downstream Task: Predicting Sepsis within 24 hours
# Target: Binary label for sepsis

data['sepsis'] = ((data['ICD_CODE'] == 'A41') & (data['CHARTTIME'] <= data['DISCHTIME'])).astype(int)

# Prepare data for training
X = data[static_features + ['avg_heart_rate', 'max_temperature', 'sentiment']]
y = data['sepsis']

# Encode categorical data
X = pd.get_dummies(X, drop_first=True)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Total features after encoding: {X_train.shape[1]}")



In [ ]:
# Deep Learning Method 1: Contrastive Learning (Siamese Network)
class SiameseNetwork(nn.Module):
    def __init__(self, input_dim):
        super(SiameseNetwork, self).__init__()
        self.shared_fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU()
        )

    def forward(self, x1, x2):
        out1 = self.shared_fc(x1)
        out2 = self.shared_fc(x2)
        return torch.abs(out1 - out2)

# Define the Siamese network
siamese_model = SiameseNetwork(X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(siamese_model.parameters(), lr=0.001)

# Train the Siamese network
for epoch in range(20):
    siamese_model.train()
    x1, x2 = torch.tensor(X_train).float(), torch.tensor(X_train[::-1]).float()
    outputs = siamese_model(x1, x2)
    loss = criterion(outputs, torch.tensor(y_train).float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



In [ ]:
# Deep Learning Method 2: Knowledge Graph-based Method with Graph Convolutional Network
print("Building Knowledge Graph for Patients...")

# Create a graph where nodes are patients, edges are based on shared diagnosis
G = nx.Graph()
for index, row in data.iterrows():
    G.add_node(row['PATIENT_ID'], age=row['age'], sentiment=row['sentiment'])
    G.add_edge(row['PATIENT_ID'], row['DIAGNOSIS'])

# Convert to PyTorch Geometric Data
edge_index = torch.tensor(list(G.edges)).t().contiguous()
node_features = torch.tensor([list(nx.get_node_attributes(G, 'age').values()),
                               list(nx.get_node_attributes(G, 'sentiment').values())]).t().float()

graph_data = Data(x=node_features, edge_index=edge_index)

# Define Graph Convolutional Network
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Train GCN with refined training loop and evaluation
print("Training GCN Model...")
gcn_model = GCN(2, 16, 1)
optimizer = torch.optim.Adam(gcn_model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(50):
    gcn_model.train()
    out = gcn_model(graph_data).squeeze()
    loss = criterion(out, torch.tensor(y_train).float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/50], Loss: {loss.item():.4f}")

# Evaluate GCN Model
with torch.no_grad():
    gcn_model.eval()
    predictions = torch.sigmoid(gcn_model(graph_data)).squeeze().round().numpy()

print("GCN Model Evaluation:")
print("Accuracy:", accuracy_score(y_train, predictions))
print("Precision:", precision_score(y_train, predictions))
print("Recall:", recall_score(y_train, predictions))
print("F1 Score:", f1_score(y_train, predictions))
print("ROC AUC:", roc_auc_score(y_train, predictions))
